In [7]:
import sys
import os

# Path to your project root
project_root = r"C:\Users\dell\Downloads\projet BDD\fhv-tripdata-mongodb-optimization"

sys.path.append(project_root)


In [8]:
from clean_data import run_cleaning_pipeline
from convert_parquet_to_json import convert_parquet_to_json
from src.mongo_import import import_json_to_mongodb
from logger import logger
#import os


l


In [28]:
import os

JSON_PATH = "../data/processed"
INPUT_PATH = "../data/raw"

if os.path.isdir(JSON_PATH):
    print("Directory exists")
else:
    print("Directory does NOT exist")


Directory exists


In [ ]:
INPUT_PATH = "../data/raw/fhvhv_tripdata_2021-10.parquet"
JSON_PATH = "../data/processed"

columns_to_remove =[
    'originating_base_num', 
    'on_scene_datetime',
    'access_a_ride_flag'
]

columns_clean = [
    'hvfhs_license_num',
    'dispatching_base_num'
]

flag_cols = [
    'shared_request_flag',
    'shared_match_flag', 
    'wav_request_flag',	
    'wav_match_flag'
]

DB_NAME = "trips_db"
COLLECTION_NAME = "fhvhv_trips_2021-10"



In [ ]:
# print(os.getcwd())
# print(os.path.exists(INPUT_PATH))

In [21]:
# Step 1: Clean the data and get a DataFrame
df = run_cleaning_pipeline(INPUT_PATH, columns_to_remove, columns_clean, flag_cols)

In [ ]:
print(df)


In [ ]:
df1 = df.iloc[0:6_000_000]

In [ ]:
l = len(df)
print(df.iloc[l-1:l])

In [ ]:
i = 0
batch_size = 1_000_000
count = 0


In [25]:
JSON_PATH = "../data/processed"
if os.path.isdir(JSON_PATH):
    print("Directory exists")
else:
    print("Directory does NOT exist")
convert_parquet_to_json(df, JSON_PATH)

Directory exists


In [ ]:
# Step 2: Convert the cleaned DataFrame to JSON Lines format
convert_parquet_to_json(df, JSON_PATH)

In [ ]:
# Step 2: Convert the cleaned DataFrame to JSON Lines format

# while i+batch_size < l:
#     batch = df.iloc[i:i+batch_size]
#     JSON_PATH = f"../data/processed/trips{count}.json"
#     convert_parquet_to_json(batch, JSON_PATH)
#     i += batch_size 
#     count +=1
# if i < l:
#     batch = df.iloc[i:l]
#     JSON_PATH = f"../data/processed/trips{count}.json"
#     convert_parquet_to_json(batch, JSON_PATH)

In [31]:
#test 1 json
JSON_PATH = "../data/processed/trips_0.json"
print()
import_json_to_mongodb(JSON_PATH, DB_NAME, COLLECTION_NAME)

In [ ]:
# Step 3: Import the JSON Lines data into MongoDB
for c in range(1,8):
    JSON_PATH = f"../data/processed/trips_{c}.json"
    import_json_to_mongodb(JSON_PATH, DB_NAME, COLLECTION_NAME)
    print(f'json{c}')
#76m 5.9s

json1
json2
json3
json4
json5
json6
json7
